# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 66 new papers today
          29 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/28 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.14322


extracting tarball to tmp_2211.14322...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:366: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

 done.


Retrieving document from  https://arxiv.org/e-print/2211.15599


extracting tarball to tmp_2211.15599... done.
Retrieving document from  https://arxiv.org/e-print/2211.15615


extracting tarball to tmp_2211.15615...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14322-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14322) | **The SPHERE view of three interacting twin disc systems in polarised  light**  |
|| Philipp Weber, et al. -- incl., <mark>Nicolás T. Kurtovic</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *23 pages, 17 figures, accepted for publication in MNRAS*|
|**Abstract**| Dense stellar environments as hosts of ongoing star formation increase the probability of gravitational encounters among stellar systems during the early stages of evolution. Stellar interaction may occur through non-recurring, hyperbolic or parabolic passages (a so-called 'fly-by'), through secular binary evolution, or through binary capture. In all three scenarios, the strong gravitational perturbation is expected to manifest itself in the disc structures around the individual stars. Here, we present near-infrared polarised light observations that were taken with the SPHERE/IRDIS instrument of three known interacting twin-disc systems: AS 205, EM* SR 24, and FU Orionis. The scattered light exposes spirals likely caused by the gravitational interaction. On a larger scale, we observe connecting filaments between the stars. We analyse their very complex polarised intensity and put particular attention to the presence of multiple light sources in these systems. The local angle of linear polarisation indicates the source whose light dominates the scattering process from the bridging region between the two stars. Further, we show that the polarised intensity from scattering with multiple relevant light sources results from an incoherent summation of the individuals' contribution. This can produce nulls of polarised intensity in an image, as potentially observed in AS 205. We discuss the geometry and content of the systems by comparing the polarised light observations with other data at similar resolution, namely with ALMA continuum and gas emission. Collective observational data can constrain the systems' geometry and stellar trajectories, with the important potential to differentiate between dynamical scenarios of stellar interaction. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14330-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14330) | **Revisiting the atmosphere of the exoplanet 51 Eridani b with VLT/SPHERE**  |
|| <mark>S. B. Brown-Sevilla</mark>, et al. -- incl., <mark>P. Mollière</mark>, <mark>M. Samland</mark>, <mark>M. Feldt</mark>, <mark>Th. Henning</mark>, <mark>M. Langlois</mark>, <mark>M. Meyer</mark>, <mark>A. Pavlov</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *Accepted for publication in A&A. 21 pages, 7 figures in the main text and 9 figures in the Appendix*|
|**Abstract**| [Full abstract in the paper] We aim to better constrain the atmospheric properties of the directly imaged exoplanet 51~Eri~b by using a retrieval approach on higher signal-to-noise data than previously reported. In this context, we also compare the results of using the atmospheric retrieval code \texttt{petitRADTRANS} vs a self-consistent model to fit atmospheric parameters. We present a higher signal-to-noise $YH$ spectrum of the planet and revised $K1K2$ photometry (M$_{K1} = 15.11 \pm 0.04$ mag, M$_{K2} = 17.11 \pm 0.38$ mag). The best-fit parameters obtained using an atmospheric retrieval differ from previous results using self-consistent models. In general, we find that our solutions tend towards cloud-free atmospheres (e.g. log $\tau_{\rm clouds} = -5.20 \pm 1.44$). For our ``nominal'' model with new data, we find a lower metallicity ([Fe/H] $= 0.26\pm$0.30 dex) and C/O ratio ($0.38\pm0.09$), and a slightly higher effective temperature (T$_{\rm{eff}} = 807\pm$45 K) than previous studies. The surface gravity (log $g = 4.05\pm0.37$) is in agreement with the reported values in the literature within uncertainties. We estimate the mass of the planet to be between 2 and 4 M$_{\rm{Jup}}$. When comparing with self-consistent models, we encounter a known correlation between the presence of clouds and the shape of the $P-T$ profiles. Our findings support the idea that results from atmospheric retrievals should not be discussed in isolation, but rather along with self-consistent temperature structures obtained using the retrieval's best-fit parameters. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14543-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14543) | **When Spectral Modeling Meets Convolutional Networks: A Method for  Discovering Reionization-era Lensed Quasars in Multi-band Imaging Data**  |
|| Irham Taufik Andika, et al. -- incl., <mark>Knud Jahnke</mark>, <mark>Sarah E. I. Bosman</mark>, <mark>Frederick B. Davies</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *24 pages, 17 figures, and 2 tables. Accepted for publication in The Astrophysical Journal. We welcome comments from the reader*|
|**Abstract**| Over the last two decades, around three hundred quasars have been discovered at $z\gtrsim6$, yet only one was identified as being strong-gravitationally lensed. We explore a new approach, enlarging the permitted spectral parameter space while introducing a new spatial geometry veto criterion, implemented via image-based deep learning. We made the first application of this approach in a systematic search for reionization-era lensed quasars, using data from the Dark Energy Survey, the Visible and Infrared Survey Telescope for Astronomy Hemisphere Survey, and the Wide-field Infrared Survey Explorer. Our search method consists of two main parts: (i) pre-selection of the candidates based on their spectral energy distributions (SEDs) using catalog-level photometry and (ii) relative probabilities calculation of being a lens or some contaminant utilizing a convolutional neural network (CNN) classification. The training datasets are constructed by painting deflected point-source lights over actual galaxy images to generate realistic galaxy-quasar lens models, optimized to find systems with small image separations, i.e., Einstein radii of $\theta_\mathrm{E} \leq 1$ arcsec. Visual inspection is then performed for sources with CNN scores of $P_\mathrm{lens} > 0.1$, which led us to obtain 36 newly-selected lens candidates, waiting for spectroscopic confirmation. These findings show that automated SED modeling and deep learning pipelines, supported by modest human input, are a promising route for detecting strong lenses from large catalogs that can overcome the veto limitations of primarily dropout-based SED selection approaches. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15224-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15224) | **The Pristine Inner Galaxy Survey (PIGS) VI: Different vertical  distributions between two DIBs at 442.8 nm and 862.1 nm**  |
|| <mark>He Zhao</mark>, et al. -- incl., <mark>Morgan Fouesneau</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *13 pages, 14 figures, accepted for publication in MNRAS*|
|**Abstract**| Although diffuse interstellar bands (DIBs) were discovered over 100 years ago, for most of them, their origins are still unknown. Investigation on the correlations between different DIBs is an important way to study the behavior and distributions of their carriers. Based on stacking thousands of spectra from the Pristine Inner Galaxy Survey, we study the correlations between two DIBs at 442.8 nm ($\lambda$442.8) and 862.1 nm ($\lambda$862.1), as well as the dust grains, in a range of latitude spanning ${\sim}22^{\circ}$ ($4^{\circ}\,{<}\,|b|\,{<}\,15^{\circ}$) toward the Galactic center ($|\ell|\,{<}\,11^{\circ}$). Tight linear intensity correlations can be found between $\lambda$442.8, $\lambda$862.1, and dust grains for $|b|\,{<}\,12^{\circ}$ or $E(B-V)\,{>}\,0.3$ mag. For $|b|\,{>}\,12^{\circ}$, $\lambda$442.8 and $\lambda$862.1 present larger relative strength with respect to the dust grains. A systematic variation of the relative strength between $\lambda$442.8 and $\lambda$862.1 with $|b|$ and $E(B-V)$ concludes that the two DIBs do not share a common carrier. Furthermore, the carrier of $\lambda$862.1 is more abundant at high latitudes than that of $\lambda$442.8. This work can be treated as an example showing the significance and potentials to the DIB research covering a large latitude range. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15570-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15570) | **GECAM Localization of High Energy Transients and the Systematic Error**  |
|| <mark>Yi Zhao</mark>, et al. -- incl., <mark>Shao-Lin Xiong</mark>, <mark>Jia-Cong Liu</mark>, <mark>Qi Liuo</mark>, <mark>Yan-Qiu Zhang</mark>, <mark>Xiao-Yun Zhao</mark>, <mark>Xiao-Bo Li</mark>, <mark>Zhen Zhang</mark>, <mark>Jin-Yuan Liao</mark>, <mark>Dong-Ya Guo</mark>, <mark>Sheng-Lun Xie</mark>, <mark>Zhi-Wei Guo</mark>, <mark>Chao-Yang Li</mark>, <mark>Jian-Jian He</mark>, <mark>Hong-Mei Zhang</mark>, <mark>Bing Li</mark>, <mark>Peng Zhang</mark>, <mark>Jing Liang</mark>, <mark>Guo-Ying Zhao</mark>, <mark>Xin-Qiao Li</mark>, <mark>Fan Zhang</mark>, <mark>Da-Li Zhang</mark>, <mark>Ya-Qing Liu</mark>, <mark>Xiao-Hua Liang</mark>, <mark>Xiao-Jing Liu</mark>, <mark>Li-Ming Song</mark>, <mark>Ke-Ke Zhang</mark>, <mark>Shu Zhang</mark>, <mark>Shuang-Nan Zhang</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *Submitted to Astrophysical Journal Supplement Series for Publication, Sep. 2022*|
|**Abstract**| Gravitational wave high-energy Electromagnetic Counterpart All-sky Monitor (GECAM) is a pair of microsatellites (i.e. GECAM-A and GECAM-B) dedicated to monitoring gamma-ray transients including gravitational waves high-energy electromagnetic counterparts, Gamma-ray Bursts, Soft Gamma-ray Repeaters, Solar Flares and Terrestrial Gamma-ray Flashes. Since launch in December 2020, GECAM-B has detected hundreds of astronomical and terrestrial events. For these bursts, localization is the key for burst identification and classification as well as follow-up observations in multi-wavelength. Here, we propose a Bayesian localization method with Poisson data with Gaussian background profile likelihood to localize GECAM bursts based on the burst counts distribution in detectors with different orientations. We demonstrate that this method can work well for all kinds of bursts, especially for extremely short ones. In addition, we propose a new method to estimate the systematic error of localization based on a confidence level test, which can overcome some problems of the existing method in literature. We validate this method by Monte Carlo simulations, and then apply it to a burst sample with accurate location and find that the mean value of the systematic error of GECAM-B localization is $\sim 2.5^{\circ}$. By considering this systematic error, we can obtain a reliable localization probability map for GECAM bursts. Our methods can be applied to other gamma-ray monitors. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15599-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15599) | **Bar formation in submaximal disks: a challenge for $Λ$CDM**  |
|| Tahere Kashfi, <mark>Mahmood Roshan</mark>, Benoit Famaey |
|*Appeared on*| *2022-11-29*|
|*Comments*| *5 pages, 4 figures; Accepted for publication in MNRAS Letters*|
|**Abstract**| Bar formation in cosmological simulations of galaxy formation remains challenging. It was previously shown that the fraction of barred galaxies at low stellar masses ($M_*<10^{10.5} M_\odot$) in TNG50 is too low compared to observations. Here, we highlight another tension, also observed at higher stellar masses, namely that barred galaxies in TNG50 appear to be maximal disks, in the sense that the majority of the gravitational acceleration is accounted for by baryons at the peak radius of the baryonic rotation curve, while observations account for a substantial fraction of barred submaximal disks. In this letter, we compare the barred fraction of submaximal disks in the local Universe from the SPARC catalogue with that in the TNG50 simulation. We show that, although SPARC tends to select against barred galaxies, the fraction of barred submaximal disks in this dataset is significantly larger than in TNG50. This result adds to the list of challenges related to predicting the right statistics and properties of barred galaxies in $\Lambda$CDM simulations of galaxy formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15615-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15615) | **Non-ideal magnetohydrodynamics of self-gravitating filaments**  |
|| Nicol Gutiérrez-Vera, et al. -- incl., <mark>Dominik R.G. Schleicher</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *11 pages, 9 figures, to be published in Astronomy & Astrophysics*|
|**Abstract**| Filaments have been studied in detail through observations and simulations. A range of numerical works have separately investigated how chemistry and diffusion effects, as well as magnetic fields and their structure impact the gas dynamics of the filament. However, non-ideal effects have hardly been explored thus far. We investigate how non-ideal magnetohydrodynamic (MHD) effects, combined with a simplified chemical model affect the evolution and accretion of a star-forming filament. We modeled an accreting self-gravitating turbulent filament using lemongrab, a one-dimensional (1D) non-ideal MHD code that includes chemistry. We explore the influence of non-ideal MHD, the orientation and strength of the magnetic field, and the cosmic ray ionization rate, on the evolution of the filament, with particular focus on the width and accretion rate. We find that the filament width and the accretion rate are determined by the magnetic field properties, including the initial strength, the coupling with the gas controlled by the cosmic ray ionization rate, and the orientation of the magnetic field with respect to the accretion flow direction. Increasing the cosmic-ray ionization rate leads to a behavior closer to that of ideal MHD, reducing the magnetic pressure support and, hence, damping the accretion efficiency with a consequent broadening of the filament width. For the same reason, we obtained a narrower width and a larger accretion rate when we reduced the initial magnetic field strength. Overall, while these factors affect the final results by approximately a factor of~2, removing the non-ideal MHD effects results in a much greater variation (up to a factor of~7). The inclusion of non-ideal MHD effects and the cosmic-ray ionization is crucial for the study of self-gravitating filaments and in determining critical observable quantities, such as the filament width and accretion rate. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.15491-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.15491) | **FIREBall-2: flight preparation of a proven balloon payload to image the  intermediate redshift circumgalactic medium**  |
|| Vincent Picouet, et al. -- incl., <mark>Zeren Lin</mark>, <mark>Jess Li</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| **|
|**Abstract**| FIREBall-2 is a stratospheric balloon-borne 1-m telescope coupled to a UV multi-object slit spectrograph designed to map the faint UV emission surrounding z~0.7 galaxies and quasars through their Lyman-alpha line emission. This spectro-imager had its first launch on September 22nd 2018 out of Ft. Sumner, NM, USA. Because the balloon was punctured, the flight was abruptly interrupted. Instead of the nominal 8 hours above 32 km altitude, the instrument could only perform science acquisition for 45 minutes at this altitude. In addition, the shape of the deflated balloon, combined with a full Moon, revealed a severe off-axis scattered light path, directly into the UV science detector and about 100 times larger than expected. In preparation for the next flight, and in addition to describing FIREBall-2's upgrade, this paper discusses the exposure time calculator (ETC) that has been designed to analyze the instrument's optimal performance (explore the instrument's limitations and subtle trade-offs). |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14329-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14329) | **First detections of stellar light from quasar host galaxies at z > 6**  |
|| Xuheng Ding, et al. -- incl., <mark>Knud Jahnke</mark>, <mark>Sarah E. I. Bosman</mark>, <mark>Melanie Habouzit</mark>, <mark>Chien-Hsiu Lee</mark>, <mark>Junyao Li</mark>, <mark>Fabian Walter</mark> |
|*Appeared on*| *2022-11-29*|
|*Comments*| *Submitted to Nature. 37 pages, 3 main figures, 1 table, 3 supplementary figures, 1 supplementary table. Comments are welcome*|
|**Abstract**| The existence of high-redshift supermassive black holes (SMBHs) -- shining brightly as quasars during the first billion years of our universe -- presents a conundrum in astrophysics. A broad variety of physical mechanisms have been proposed for the formation and rapid growth of these early SMBHs. Promising diagnostics are the relative properties of the black hole and its host galaxy. However, up to now, the detection of stars in quasar host galaxies has been elusive beyond $z>2$, even with deep HST observations. Here, we report the first detections of the stellar component of the host galaxies of two relatively low-luminosity quasars at $z>6$ observed with JWST using NIRCam. After modeling and subtracting the glare from the quasar itself, we find that the host galaxies are massive (stellar mass of $2.5\times$ and $6.3\times10^{10}$ M$_{\odot}$), compact, and disk-like. Unlike most SMBHs in the nearby universe, these quasars are displaced from the centers of their host galaxies in the rest-frame optical, in one case by $0.9$ kpc. These first positive detections of quasar hosts at $z>6$ are a pivotal milestone; we can now assess the stellar environment along with star formation and black hole accretion to determine the physical conditions that govern the formation and evolution of the first SMBHs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.14322.md
    + _build/html/tmp_2211.14322/./Figures/Figure2.png
    + _build/html/tmp_2211.14322/./Figures/Figure5.png
    + _build/html/tmp_2211.14322/./Figures/Figure10.png
exported in  _build/html/2211.14330.md
    + _build/html/tmp_2211.14330/./figures/TelluricAbsportionIFS_1p4mu_20170928.png
    + _build/html/tmp_2211.14330/./figures/SkyIRtemp_LHATPRO_20170928.png
    + _build/html/tmp_2211.14330/./figures/PWV_LHATPRO_20170928.png
    + _build/html/tmp_2211.14330/./figures/spec_new_clouds.png
    + _build/html/tmp_2211.14330/./figures/corner_new_clouds.png
exported in  _build/html/2211.14543.md
    + _build/html/tmp_2211.14543/figures/fig_roc_curve_sie.png
    + _build/html/tmp_2211.14543/figures/fig_qso_gal_synspec.png
    + _build/html/tmp_2211.14543/figures/fig_lens_color.png
exported in  _build/html/2211.15224.md
    + _build/html/tmp_2211.15224/./figures/bGroup.png
    + _build/html/tmp_2211.15224/./figures/ratio-babs.png
    + _build/html/tmp_22

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The SPHERE view of three interacting twin disc systems in polarised light

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.14322-b31b1b.svg)](https://arxiv.org/abs/2211.14322)<mark>Appeared on: 2022-11-29</mark> - _23 pages, 17 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

Philipp Weber, et al. -- incl., <mark><mark>Nicolás T. Kurtovic</mark></mark>

</div>
<div id="abstract">

**Abstract:** Dense stellar environments as hosts of ongoing star formation increase the probability of gravitational encounters among stellar systems during the early stages of evolution. Stellar interaction may occur through non-recurring, hyperbolic or parabolic passages (a so-called ‘fly-by'), through secular binary evolution, or through binary capture. In all three scenarios, the strong gravitational perturbation is expected to manifest itself in the disc structures around the individual stars.Here, we present near-infrared polarised light observations that were taken with the SPHERE/IRDIS instrument of three known interacting twin-disc systems: AS 205, EM* SR 24, and FU Orionis.The scattered light exposes spirals likely caused by the gravitational interaction.On a larger scale, we observe connecting filaments between the stars.We analyse their very complex polarised intensity and put particular attention to the presence of multiple light sources in these systems. The local angle of linear polarisation indicates the source whose light dominates the scattering process from the bridging region between the two stars.Further, we show that the polarised intensity from scattering with multiple relevant light sources results from an incoherent summation of the individuals' contribution.This can produce nulls of polarised intensity in an image, as potentially observed in AS 205.We discuss the geometry and content of the systems by comparing the polarised light observations with other data at similar resolution, namely with ALMA continuum and gas emission.Collective observational data can constrain the systems' geometry and stellar trajectories, with the important potential to differentiate between dynamical scenarios of stellar interaction.

</div>

<div id="div_fig1">

<img src="tmp_2211.14322/./Figures/Figure2.png" alt="Fig10" width="100%"/>

**Figure 10. -** SPHERE/IRDIS observations of three twin disc systems: AS 205, SR 24 and FU Orionis. North is up, East is to the left. We show the normalised polarised light intensity in logarithmic colour scale with the $1\sigma_{\rm rms}-$level as the lower limit and the maximum intensity as the upper limit. The coordinates are centred on the respective northern source. The central circles mark the coronagraph's diameter. The location of the companions in the images are marked by a star. The length of the bar in the lower left corner of each panel is equivalent to a projected distance of 100$ $au. Arrows and letters are referred to in the text. For SR 24 the southern source is very close to the edge of the detector which is why the lower frame is visible there. (*fig:mosaic*)

</div>
<div id="div_fig2">

<img src="tmp_2211.14322/./Figures/Figure5.png" alt="Fig4" width="100%"/>

**Figure 4. -** Lower panel: ALMA band 6 intensity map for SR 24. Contours trace the $H$-Band polarised intensity presented in Fig. \ref{fig:mosaic} for comparison at levels of 0.25\%, 1\% and 5\% of the peak intensity. The bar in the bottom right corner shows the projected distance of 100$ $au. The inset zooms in on the location of SR 24N. Contrast in the inset is increased by a factor of ten, the two sources SR 24Na and SR 24Nb are detected with $\sim12 \sigma_{\rm rms}$ and $\sim 5 \sigma_{\rm rms}$, respectively. The bar in the top left corner of the inset corresponds to 10$ $au. Top panel: Intensity along the connecting line between SR 24Na and SR 24Nb, north-east is to the left, south-west to the right. The circles show the ALMA band 6 data points, the dashed lines show the beam size of the observation. (*fig:cont_sr24*)

</div>
<div id="div_fig3">

<img src="tmp_2211.14322/./Figures/Figure10.png" alt="Fig12" width="100%"/>

**Figure 12. -** {The left and central panel show dynamical stellar mass estimates for AS 205N and AS 205S, respectively, measured with the {\tt eddy}-package \citep[][]{eddy}. The stellar masses were calculated for different assumed disc inclinations, spread around the estimates provided in \citet{Kurtovic2018} from different methods, for which the measured inclination values are indicated by vertical dashed lines. At each assumed inclination we varied the size of the mask to which the dynamical measurement was limited between radii of $0$\farcs$1$, $0$\farcs$2$, $0$\farcs$3$ and $0$\farcs$4$. AS a visual example, the right panel shows the moment 1 map centred on AS 205N with the different mask sizes indicated by dashed circles. The colour of the data points in the two left panels corresponds to the colour of the applied mask in the right panel.} (*fig:AS_kine_masks*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\SubItem}[1]{$
$    {\setlength\itemindent{15pt} \item[-] #1}$
$}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{\SubItem}[1]{$
$    {\setlength\itemindent{15pt} \item[-] #1}$
$}$
$\newcommand{$\arraystretch$}{1.5}$
$\newcommand{$\arraystretch$}{1.5}$</div>



<div id="title">

# Revisiting the atmosphere of the exoplanet 51 Eridani b with VLT/SPHERE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.14330-b31b1b.svg)](https://arxiv.org/abs/2211.14330)<mark>Appeared on: 2022-11-29</mark> - _Accepted for publication in A&A. 21 pages, 7 figures in the main text and 9 figures in the Appendix_

</div>
<div id="authors">

<mark><mark>S. B. Brown-Sevilla</mark></mark>, et al. -- incl., <mark><mark>P. Mollière</mark></mark>, <mark><mark>M. Samland</mark></mark>, <mark><mark>M. Feldt</mark></mark>, <mark><mark>Th. Henning</mark></mark>, <mark><mark>M. Langlois</mark></mark>, <mark><mark>M. Meyer</mark></mark>, <mark><mark>A. Pavlov</mark></mark>

</div>
<div id="abstract">

**Abstract:** [Full abstract in the paper] We aim to better constrain the atmospheric properties of the directly imaged exoplanet 51~Eri~b by using a retrieval approach on higher signal-to-noise data than previously reported. In this context, we also compare the results of using the atmospheric retrieval code \texttt{petitRADTRANS} vs a self-consistent model to fit atmospheric parameters. We present a higher signal-to-noise $YH$ spectrum of the planet and revised $K1K2$ photometry (M$_{K1} = 15.11 \pm 0.04$ mag, M$_{K2} = 17.11 \pm 0.38$ mag). The best-fit parameters obtained using an atmospheric retrieval differ from previous results using self-consistent models. In general, we find that our solutions tend towards cloud-free atmospheres (e.g. log $\tau_{\rm clouds} = -5.20 \pm 1.44$). For our ``nominal'' model with new data, we find a lower metallicity ([Fe/H] $= 0.26\pm$0.30 dex) and C/O ratio ($0.38\pm0.09$), and a slightly higher effective temperature (T$_{\rm{eff}} = 807\pm$45 K) than previous studies. The surface gravity (log $g = 4.05\pm0.37$) is in agreement with the reported values in the literature within uncertainties. We estimate the mass of the planet to be between 2 and 4 M$_{\rm{Jup}}$. When comparing with self-consistent models, we encounter a known correlation between the presence of clouds and the shape of the $P-T$ profiles. Our findings support the idea that results from atmospheric retrievals should not be discussed in isolation, but rather along with self-consistent temperature structures obtained using the retrieval's best-fit parameters. 

</div>

<div id="div_fig1">

<img src="tmp_2211.14330/./figures/TelluricAbsportionIFS_1p4mu_20170928.png" alt="Fig6.1" width="33%"/><img src="tmp_2211.14330/./figures/SkyIRtemp_LHATPRO_20170928.png" alt="Fig6.2" width="33%"/><img src="tmp_2211.14330/./figures/PWV_LHATPRO_20170928.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** Top: Telluric absorption as traced by SPHERE/IFS at $\approx 1.4\mu$m. Middle: Sky infrared temperature as traced by LHATPRO. Bottom: PWV as traced by LHATPRO.  The vertical dashed line in all figures marks our cut-off time for the first 140 IFS frames, which form the basis of our spectral analysis, out of a total of 154 IFS data frames. (*fig:telluric_timeseries*)

</div>
<div id="div_fig2">

<img src="tmp_2211.14330/./figures/spec_new_clouds.png" alt="Fig11" width="100%"/>

**Figure 11. -** \texttt{petitRADTRANS} best-fit spectrum of 51 Eri b for the "enforced clouds" retrieval run on our new SPHERE spectro-photometric data (teal circles and purple squares) along with the photometric points included in \cite{Samland2017}(shown as squares). The photometric points describe the average flux in the respective filter, the \textit{x}-error bar represents the filter widths. 34 randomly drawn samples from the posterior probability distribution are shown in gray, to show the spread of model parameter combinations to fit the data. Residuals in multiples of 1 $\sigma$ uncertainties of the data are shown below. (*fig:new_cloudy*)

</div>
<div id="div_fig3">

<img src="tmp_2211.14330/./figures/corner_new_clouds.png" alt="Fig12" width="100%"/>

**Figure 12. -** Corner plot of the posterior PDFs of the "enforced clouds" retrieval run on the new data set. (*fig:cloudy_corner*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.14543-b31b1b.svg)](https://arxiv.org/abs/2211.14543)<mark>Appeared on: 2022-11-29</mark> - _24 pages, 17 figures, and 2 tables. Accepted for publication in The Astrophysical Journal. We welcome comments from the reader_

</div>
<div id="authors">

Irham Taufik Andika, et al. -- incl., <mark>Knud Jahnke</mark>, <mark>Sarah E. I. Bosman</mark>, <mark>Frederick B. Davies</mark>

</div>
<div id="abstract">

**Abstract:** Over the last two decades, around three hundred quasars have been discovered at$z\gtrsim6$, yet only one was identified as being strong-gravitationally lensed.We explore a new approach, enlarging the permitted spectral parameter space while introducing a new spatial geometry veto criterion, implemented via image-based deep learning.We made the first application of this approach in a systematic search for reionization-era lensed quasars, using data from the Dark Energy Survey, the Visible and Infrared Survey Telescope for Astronomy Hemisphere Survey, and the Wide-field Infrared Survey Explorer.Our search method consists of two main parts:(i) pre-selection of the candidates based on their spectral energy distributions (SEDs) using catalog-level photometry and(ii) relative probabilities calculation of being a lens or some contaminant utilizing a convolutional neural network (CNN) classification.The training datasets are constructed by painting deflected point-source lights over actual galaxy images to generate realistic galaxy-quasar lens models, optimized to find systems with small image separations, i.e., Einstein radii of$\theta_\mathrm{E} \leq 1$\arcsec$$.Visual inspection is then performed for sources with CNN scores of$P_\mathrm{lens} > 0.1$, which led us to obtain 36 newly-selected lens candidates, waiting for spectroscopic confirmation.These findings show that automated SED modeling and deep learning pipelines, supported by modest human input, are a promising route for detecting strong lenses from large catalogs that can overcome the veto limitations of primarily dropout-based SED selection approaches.

</div>

<div id="div_fig1">

<img src="tmp_2211.14543/figures/fig_roc_curve_sie.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
		Receiver operating characteristic (ROC) curve and the associated area under the curve (AUC) value.
		The curve for classifying lensed quasars is presented with a solid blue line.
		On the other hand, the curves for predicting galaxies, MLT dwarfs, and quasars are shown with magenta, yellow, and cyan dashed lines, respectively.
		The FPR and TPR value for the adopted $P_\mathrm{lens}$ thresholds is also shown as a red circle.
	 (*fig:roc_curve*)

</div>
<div id="div_fig2">

<img src="tmp_2211.14543/figures/fig_qso_gal_synspec.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
		Example of the simulated galaxy-quasar lens spectrum (gray line) and the associated synthetic photometry (red circles).
		The unlensed background quasar's emission (orange line) is magnified with a factor of $\mu$ before being added to the foreground galaxy's emission (blue line) to produce the lensed quasar spectrum.
	 (*fig:qso_gal_synspec*)

</div>
<div id="div_fig3">

<img src="tmp_2211.14543/figures/fig_lens_color.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
		Color diagram of $Y_\mathrm{DES} - J_\mathrm{VHS}$ vs. $z_\mathrm{DES} - Y_\mathrm{DES}$.
		The colors of mock quasars (black circles), low-$z$ early-type galaxies (orange circles), simulated lenses (red circles), and a sample of MLT dwarfs \citep[blue circles;][]{2018ApJS..234....1B,2019MNRAS.489.5301C} are shown in the figure.
		Our color cuts pre-selection is marked by the dashed black box, which focuses on finding lensed quasars located at $z=6.6$--7.2.
		Note that the seemingly unnatural rectangle shape of the MLT dwarfs distribution on the right of the plot is caused by the color cut criteria used by \cite{2019MNRAS.489.5301C} to avoid the quasar color locus.
	 (*fig:lens_color*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\EBV}{E(B {-} V)}$
$\newcommand{\Teff}{T_{\rm eff}}$
$\newcommand{\logg}{{\rm log} g}$
$\newcommand{\feh}{\rm[Fe/H]}$
$\newcommand{\kms}{\rm km s^{-1}}$
$\newcommand{\Mathias}[1]{{\color{magenta} [Mathias:#1]}}$
$\newcommand{\hz}[1]{{\color{red} [He:#1]}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\EBV$}{E(B {-} V)}$
$\newcommand{$\Teff$}{T_{\rm eff}}$
$\newcommand{$\logg$}{{\rm log} g}$
$\newcommand{$\feh$}{\rm[Fe/H]}$
$\newcommand{$\kms$}{\rm km s^{-1}}$
$\newcommand{$\Mathias$}[1]{{\color{magenta} [Mathias:#1]}}$
$\newcommand{$\hz$}[1]{{\color{red} [He:#1]}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The Pristine Inner Galaxy Survey (PIGS) VI: Different verticaldistributions between two DIBs at 442.8 nm and 862.1 nm

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.15224-b31b1b.svg)](https://arxiv.org/abs/2211.15224)<mark>Appeared on: 2022-11-29</mark> - _13 pages, 14 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

<mark><mark>He Zhao</mark></mark>, et al. -- incl., <mark><mark>Morgan Fouesneau</mark></mark>

</div>
<div id="abstract">

**Abstract:** Although diffuse interstellar bands (DIBs) were discovered over 100 years ago, for most of them, their origins are still unknown.Investigation on the correlations between different DIBs is an important way to study the behavior and distributions of their carriers.Based on stacking thousands of spectra from the Pristine Inner Galaxy Survey, we study the correlations between two DIBs at 442.8 nm($\lambda$442.8) and 862.1 nm ($\lambda$862.1), as well as the dust grains, in a range of latitude spanning${\sim}22^{\circ}$($4\degr{<} |b| {<} 15\degr$) toward the Galactic center ($|\ell| {<} 11^{\circ}$). Tight linear intensity correlationscan be found between$\lambda$442.8,$\lambda$862.1, and dust grains for$|b| {<} 12^{\circ}$or$\EBV{>} 0.3$mag.For$|b| {>} 12^{\circ}$,$\lambda$442.8 and$\lambda$862.1 present larger relative strength with respect to the dust grains.A systematic variation of the relative strength between$\lambda$442.8 and$\lambda$862.1 with$|b|$and$\EBV$concludes that thetwo DIBs do not share a common carrier. Furthermore, the carrier of$\lambda$862.1 is more abundant at high latitudes than that of$\lambda$442.8. This work can be treated as an example showing the significance and potentials to the DIB research covering alarge latitude range.

</div>

<div id="div_fig1">

<img src="tmp_2211.15224/./figures/bGroup.png" alt="Fig5" width="100%"/>

**Figure 5. -** Spatial distribution $(\ell,b)$ of 29 PIGS fields, overplotted on the dust reddening map of \citet{Planck2016dust}.
    Different colors indicate different latitude groups defined in Sect. \ref{subsect:group}, that is red: G1, $|b| {>} 12^{\circ}$;
    yellow: G2, $8^{\circ} {<} |b| {<} 12^{\circ}$ and $\ell{>}{-}1^{\circ}$; cyan: G3, $8^{\circ} {<} |b| {<} 12^{\circ}$
    and $\ell{<}{-}1^{\circ}$, and blue: G4, $|b| {<} 8^{\circ}$. The median $\EBV$ of each filed is also indicated. (*fig:bgroup*)

</div>
<div id="div_fig2">

<img src="tmp_2211.15224/./figures/ratio-babs.png" alt="Fig8" width="100%"/>

**Figure 8. -** Variation of relative strength between DIBs $\lambda$442.8, $\lambda$862.1, and dust with Galactic latitude ($|b|$). The
  dashed green lines indicate their average strength ratios from linear fits (see Sect. \ref{subsect:linear}). See Sect.
  \ref{subsect:group} and Fig. \ref{fig:bgroup} for the point colors representing different latitude groups. (*fig:var-b*)

</div>
<div id="div_fig3">

<img src="tmp_2211.15224/./figures/ratio-EBV.png" alt="Fig9" width="100%"/>

**Figure 9. -** The same as Fig. \ref{fig:var-b}, but for the variation with $\EBV$. (*fig:var-ebv*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\logg}{\log g }$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\fehphot}{[Fe/H]_{\mathrm{phot}}}$
$\newcommand{\alphafe}{[\alpha/Fe]}$
$\newcommand{\rproj}{R_{\mathrm{proj}}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\psub}{p_{\mathrm{sub}}}$
$\newcommand{\meanfeh}{\langle\feh \rangle}$
$\newcommand{\meanalphafe}{\langle\alphafe \rangle}$
$\newcommand{\sigmaalphafe}{\sigma\alphafe}$
$\newcommand{\sigmafeh}{\sigma\feh}$
$\newcommand{\gradientfehism}{-0.0048\pm0.0003}$
$\newcommand{\gradientfehicsm}{-0.0075\pm0.0003}$
$\newcommand{\gradientfehiss}{-0.0070\pm0.0007}$
$\newcommand{\gradientfehicss}{-0.0050\pm0.0003}$
$\newcommand{\gradientfehicsms}{-0.0050\pm0.0003}$
$\newcommand{\gradientfehicsmslimited}{-0.0094\pm0.0003}$
$\newcommand{\gradientalphaism}{+0.0040\pm0.0006}$
$\newcommand{\gradientalphaicsm}{+0.0020\pm0.0003}$
$\newcommand{\gradientalphaiss}{+0.0001\pm0.0008}$
$\newcommand{\gradientalphaicss}{+0.0002\pm0.0006}$
$\newcommand{\gradientalphaicsms}{+0.0026\pm0.0004}$
$\newcommand{\gradientalphaicsmlimited}{+0.0055\pm0.0020}$
$\newcommand{\gradientalphaicsmslimited}{+0.0030\pm0.0017}$
$\newcommand{\meanfehGSS}{-0.859\pm0.004}$
$\newcommand{\meanalphafeGSS}{0.107\pm0.017}$
$\newcommand{\meanfehGSSsmooth}{-1.080\pm0.005}$
$\newcommand{\meanalphafeGSSsmooth}{0.340\pm0.012}$
$\newcommand{\meanfehGSSenv}{-1.459\pm0.011}$
$\newcommand{\meanalphafeGSSenv}{0.066\pm0.026}$
$\newcommand{\meanfehGSSenvsmooth}{-1.391\pm0.013}$
$\newcommand{\meanalphafeGSSenvsmooth}{0.670\pm0.032}$
$\newcommand{\meanfehSE}{-0.927\pm0.005}$
$\newcommand{\meanalphafeSE}{0.063\pm0.017}$
$\newcommand{\meanfehSEsmooth}{-0.949\pm0.007}$
$\newcommand{\meanalphafeSEsmooth}{0.357\pm0.018}$
$\newcommand{\meanfehStC}{-1.313\pm0.007}$
$\newcommand{\meanalphafeStC}{0.199\pm0.020}$
$\newcommand{\meanfehStCsmooth}{-1.599\pm0.007}$
$\newcommand{\meanalphafeStCsmooth}{0.394\pm0.018}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\logg$}{\log g }$
$\newcommand{$\teff$}{T_\mathrm{eff}}$
$\newcommand{$\feh$}{[Fe/H]}$
$\newcommand{$\feh$phot}{[Fe/H]_{\mathrm{phot}}}$
$\newcommand{$\alphafe$}{[\alpha/Fe]}$
$\newcommand{$\rproj$}{R_{\mathrm{proj}}}$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand{$\psub$}{p_{\mathrm{sub}}}$
$\newcommand{$\meanfeh$}{\langle$\feh$ \rangle}$
$\newcommand{$\meanalphafe$}{\langle$\alphafe$ \rangle}$
$\newcommand{$\sigmaalphafe$}{\sigma$\alphafe$}$
$\newcommand{$\sigmafeh$}{\sigma$\feh$}$
$\newcommand{$\gradientfehism$}{-0.0048\pm0.0003}$
$\newcommand{$\gradientfehicsm$}{-0.0075\pm0.0003}$
$\newcommand{$\gradientfehiss$}{-0.0070\pm0.0007}$
$\newcommand{$\gradientfehicss$}{-0.0050\pm0.0003}$
$\newcommand{$\gradientfehicsm$s}{-0.0050\pm0.0003}$
$\newcommand{$\gradientfehicsm$slimited}{-0.0094\pm0.0003}$
$\newcommand{$\gradientalphaism$}{+0.0040\pm0.0006}$
$\newcommand{$\gradientalphaicsm$}{+0.0020\pm0.0003}$
$\newcommand{$\gradientalphaiss$}{+0.0001\pm0.0008}$
$\newcommand{$\gradientalphaicss$}{+0.0002\pm0.0006}$
$\newcommand{$\gradientalphaicsm$s}{+0.0026\pm0.0004}$
$\newcommand{$\gradientalphaicsm$limited}{+0.0055\pm0.0020}$
$\newcommand{$\gradientalphaicsm$slimited}{+0.0030\pm0.0017}$
$\newcommand{$\meanfeh$GSS}{-0.859\pm0.004}$
$\newcommand{$\meanalphafe$GSS}{0.107\pm0.017}$
$\newcommand{$\meanfeh$GSSsmooth}{-1.080\pm0.005}$
$\newcommand{$\meanalphafe$GSSsmooth}{0.340\pm0.012}$
$\newcommand{$\meanfeh$GSSenv}{-1.459\pm0.011}$
$\newcommand{$\meanalphafe$GSSenv}{0.066\pm0.026}$
$\newcommand{$\meanfeh$GSSenvsmooth}{-1.391\pm0.013}$
$\newcommand{$\meanalphafe$GSSenvsmooth}{0.670\pm0.032}$
$\newcommand{$\meanfeh$SE}{-0.927\pm0.005}$
$\newcommand{$\meanalphafe$SE}{0.063\pm0.017}$
$\newcommand{$\meanfeh$SEsmooth}{-0.949\pm0.007}$
$\newcommand{$\meanalphafe$SEsmooth}{0.357\pm0.018}$
$\newcommand{$\meanfeh$StC}{-1.313\pm0.007}$
$\newcommand{$\meanalphafe$StC}{0.199\pm0.020}$
$\newcommand{$\meanfeh$StCsmooth}{-1.599\pm0.007}$
$\newcommand{$\meanalphafe$StCsmooth}{0.394\pm0.018}$</div>



<div id="title">

#  throughout the M31 Halo with SPLASH

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.15288-b31b1b.svg)](https://arxiv.org/abs/2211.15288)<mark>Appeared on: 2022-11-29</mark> - _Submitted to ApJ_

</div>
<div id="authors">

<mark>J. Leigh Wojno</mark>, et al.

</div>
<div id="abstract">

**Abstract:** We present spectroscopic chemical abundances of red giant branch (RGB) stars in Andromeda (M31), using medium resolution ($R\sim6000$) spectra obtained via the Spectroscopic and Photometric Landscape of Andromeda's Stellar Halo (SPLASH) survey. In addition to individual chemical abundances, we coadd low signal-to-noise ratio (S/N) spectra of stars to obtain a high enough to measure average$\feh$and$\alphafe$abundances. We obtain individual and coadded measurements for$\feh$and$\alphafe$for M31 halo stars, covering a range of 9--180 kpc in projected radius from the center of M31. With these measurements, we greatly increase the number of outer halo ($\rproj$$> 50$kpc) M31 stars with spectroscopic$\feh$and$\alphafe$, adding abundance measurements for 45 individual stars and 33 coadds from a pool of an additional 174 stars. We measure the spectroscopic metallicity ($\feh$) gradient, finding a negative radial gradient of$-0.0050\pm0.0003$for all stars in the halo, consistent with gradient measurements obtained using photometric metallicities. Using the first measurements of$\alphafe$for M31 halo stars covering a large range of projected radii, we find a positive gradient ($+0.0026\pm0.0004$) in$\alphafe$as a function of projected radius. We also explore the distribution in$\feh$--$\alphafe$space as a function of projected radius for both individual and coadded measurements in the smooth halo, and compare these measurements to those stars potentially associated with substructure. These spectroscopic abundance distributions highlight the substantial evidence that M31 has had an appreciably different formation and merger history compared to our own Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2211.15288/./figures/feh_gradient_all.png" alt="Fig7" width="100%"/>

**Figure 7. -** Spectroscopic $\feh$ versus projected radius for stars with individual/coadded (filled/open circles) $\feh$ measurements.  Grey points are photometric $\feh$ measurements for our sample. Top left: The best-fit spectroscopic $\feh$ gradient for individual smooth halo stars (Section \ref{sec:feh_smooth_halo}, solid black), where the shaded regions indicate a 1$\sigma$ standard deviation. The red dotted line represents the gradient measured by \citet{gilbert14} for smooth halo stars, using photometric $\feh$. The magenta line shows the best-fit gradient calculated using both individual stars and coadds (Section \ref{sec:coadded_abundances}). Bottom left: Individual and coadded (filled/open circles) $\feh$ measurements, but for a limited radial range ($\rproj$$< 35$ kpc). The sample of \citet{escala20b} is shown as light blue points, and the gradient measured with their sample is shown as a dashed blue line. The solid magenta line is the same as the top-left panel. We also show the fit after limiting our sample to the same radial range (dashed magenta). Top right: Individual/coadded (filled/open dark blue squares) $\feh$ measurements for stars potentially associated with substructure (Section \ref{sec:substructure_probability}), with the best-fit gradient to individual and coadded measurements from the substructure sample (cyan), and the gradient including all stars (both the smooth halo and substructure samples; blue). The photometric $\feh$ gradient from \citet{gilbert14}, fit using smooth halo and substructure stars, is shown as a dashed purple line. Bottom right: The same as above, but for a limited range ($\rproj$$<90$kpc). Regardless of our sample selection, we recover a statistically significant metallicity gradient in the halo of M31. (*fig:metallicity_gradient_individ*)

</div>
<div id="div_fig2">

<img src="tmp_2211.15288/./figures/substr_comparison_gss.png" alt="Fig11" width="100%"/>

**Figure 11. -** Top left: Histogram of the radial velocities for smooth halo stars with secure abundances in the radial range $15 <$\rproj$< 50$ kpc (grey) and stars associated with substructure in fields on the GSS core (f207, f2, a3; green). The vertical lines represent the mean line-of-sight velocity for the GSS (purple),
    and the secondary kinematically cold component (light green, Section \ref{sec:substructure_gss}) for each field shown \citep[][see their Table 4]{gilbert18}. The shaded regions indicate the corresponding velocity dispersion.
    The systemic velocity ($-300.0$\kms) of M31 is in grey. Top right: histograms of spectroscopic $\feh$ for the smooth halo (grey), individual (15 stars; solid green) and coadded stars potentially associated with substructure in these fields (27 coadds representing 136 stars; dashed green). Bottom left: Spectroscopic $\feh$ as a function of line-of-sight velocity for substructure stars (outlined green symbols) and smooth halo stars (grey). We show the same mean and dispersion velocity for the GSS and KCC as in the top left panel.
    Bottom right: $\feh$\--$\alphafe$ measurements for individual substructure stars (open green symbols), as well as coadded groupings of stars (filled green symbols).
    We also show the $\feh$\--$\alphafe$ distribution of smooth halo stars in grey. Solid lines show the mean $\feh$ and $\alphafe$ for the entire substructure sample (green), and the smooth halo (grey). (*fig:substr_comparison_gss*)

</div>
<div id="div_fig3">

<img src="tmp_2211.15288/./figures/alphafe_gradient_all.png" alt="Fig8" width="100%"/>

**Figure 8. -** The same as Figure \ref{fig:metallicity_gradient_individ}, but for the abundance gradient with respect to $\alphafe$. Top left: The best-fit spectroscopic $\alphafe$ gradient fit to individual and coadded abundance measurements is shown as a solid magenta line, with the shaded regions indicating a 1$\sigma$ standard deviation. The gradient fit using only individual measurements is shown as a black line. Bottom left: Zoomed in version of the top panel, now showing spectroscopic measurements from \citet{escala20b}(blue points), with their gradient (blue dashed line). Using the same radial range as in \citet{escala20b}, we find a gradient of $\gradientalphaicsm$limited dex kpc$^{-1}$(magenta dashed line). Top right: The best-fit gradient for individual and coadded stars associated with substructure is shown as the light blue line, with the gradient measured for all stars (individual and coadded, smooth halo and substructure) shown in dark blue. Bottom right: Zoomed in version of the top right panel, showing the best-fit gradient of all stars restricted to the same radial range as in \citet{escala20b}($\gradientalphaicsm$slimited dex kpc$^{-1}$, blue dotted line). (*fig:alphafe_gradient*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\angstrom}{\mbox{\normalfont\AA}}$
$\newcommand{\todo}[1]{\textcolor{red}{#1}}$
$\newcommand{\verify}[1]{\textcolor{violet}{#1}}$
$\newcommand{\ok}[1]{\textcolor{green}{#1}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\MJ}{M_\mathrm{J}}$
$\newcommand{\Mp}{M_\mathrm{p}}$
$\newcommand{\Macc}{\dot{M}_\mathrm{acc}}$
$\newcommand{\RJ}{R_\mathrm{J}}$
$\newcommand{\Rp}{R_\mathrm{p}}$
$\newcommand{\Teff}{T_{\text{eff}}}$
$\newcommand{\MJyr}{\MJ \mathrm{yr}^{-1}}$
$\newcommand{\Rd}{R_\mathrm{dust}}$
$\newcommand{\Rg}{R_\mathrm{gas}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Lacc}{L_\mathrm{acc}}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.45}$
$\newcommand{\arraystretch}{1.25}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\angstrom$}{\mbox{\normalfont\AA}}$
$\newcommand{$\todo$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\verify$}[1]{\textcolor{violet}{#1}}$
$\newcommand{$\ok$}[1]{\textcolor{green}{#1}}$
$\newcommand{$\Lsun$}{L_\odot}$
$\newcommand{$\Msun$}{M_\odot}$
$\newcommand{$\MJ$}{M_\mathrm{J}}$
$\newcommand{$\Mp$}{M_\mathrm{p}}$
$\newcommand{$\Macc$}{\dot{M}_\mathrm{acc}}$
$\newcommand{$\RJ$}{R_\mathrm{J}}$
$\newcommand{$\Rp$}{R_\mathrm{p}}$
$\newcommand{$\Teff$}{T_{\text{eff}}}$
$\newcommand{$\MJ$yr}{$\MJ$ \mathrm{yr}^{-1}}$
$\newcommand{$\Rd$}{R_\mathrm{dust}}$
$\newcommand{$\Rg$}{R_\mathrm{gas}}$
$\newcommand{$\micron$}{\mu\mathrm{m}}$
$\newcommand{$\Lacc$}{L_\mathrm{acc}}$
$\newcommand{$\arraystretch$}{1.25}$
$\newcommand{$\arraystretch$}{1.45}$
$\newcommand{$\arraystretch$}{1.25}$</div>



<div id="title">

# The reduced images and contrast curves are only available at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via https://cdsarc.u-strasbg.fr/viz-bin/qcat?J/A+A/

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.15434-b31b1b.svg)](https://arxiv.org/abs/2211.15434)<mark>Appeared on: 2022-11-29</mark> - _25 pages, 16 figures, 3 tables, accepted for publication in A&A_

</div>
<div id="authors">

Gabriele Cugno, et al. -- incl., <mark><mark>Ralf Launhardt</mark></mark>, <mark><mark>Thomas Henning</mark></mark>, <mark><mark>André Müller</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present the statistical analysis of a subsample of 45 young stars surrounded by protoplanetary disks (PPDs). This is the largest imaging survey uniquely focused on PPDs to date. Our goal is to search for young forming companions embedded in the disk material and to constrain their occurrence rate in relation to the formation mechanism. We used principal component analysis based point spread function subtraction techniques to reveal young companions forming in the disks. We calculated detection limits for our datasets and adopted a black-body model to derive temperature upper limits of potential forming planets. We then used Monte Carlo simulations to constrain the population of forming gas giant companions and compare our results to different types of formation scenarios. Our data revealed a new binary system (HD38120) and a recently identified triple system with a brown dwarf companion orbiting a binary system (HD101412), in addition to 12 known companions. Furthermore, we detected signals from 17 disks, two of which (HD72106 and TCrA) were imaged for the first time. We reached median detection limits of L =15.4 mag at 2.0 arcsec, which were used to investigate the temperature of potentially embedded forming companions. We can constrain the occurrence of forming planets with semi-major axis a in [20 - 500] au and Teff in [600 - 3000] K, in line with the statistical results obtained for more evolved systems from other direct imaging surveys. The NaCo-ISPY data confirm that massive bright planets accreting at high rates are rare. More powerful instruments with better sensitivity in the near- to mid-infrared are likely required to unveil the wealth of forming planets sculpting the observed disk substructures. 

</div>

<div id="div_fig1">

<img src="tmp_2211.15434/./System_properties.png" alt="Fig1" width="100%"/>

**Figure 1. -** Stellar and disk parameters for the objects in the ISPY PPD sample. In the top row the histogram for the distances (left) and $L'$ observed magnitudes (right) are reported. Stellar effective temperatures (left) and masses (right) are shown in the middle row. Finally, the bottom row provides the distributions of the disk outer radius (gas and mm-dust) on the left and disk inclinations (whenever measured) on the right. The dashed vertical lines represent the distance cutoff applied in Sect. \ref{sec:additional_selection}({\it top left panel}) and the limiting brightness for the coronagraphic observations ({\it top right panel}), respectively. (*fig:system_properties*)

</div>
<div id="div_fig2">

<img src="tmp_2211.15434/./Completeness_all.png" alt="Fig11" width="100%"/>

**Figure 11. -** Depth of search for the 45 ISPY-PPD targets included in the analysis, reporting the number of stars to which the survey is complete for young forming planets in the $\Teff-a$ parameter space. The three plots represent maps when assuming $\Rp=2,3,5 $\RJ$$({\it left, mid} and {\it right} panels, respectively). Overplotted as full black marker the companion HD101412 B.  (*fig:prob_map*)

</div>
<div id="div_fig3">

<img src="tmp_2211.15434/./Completeness_extinction.png" alt="Fig16" width="100%"/>

**Figure 16. -** Survey completeness maps obtained considering an additional extinction factor $A_{L'}=1.0$ mag. The three plots represent maps when assuming $\Rp=2,3,5 $\RJ$$({\it left, middle} and {\it right} panels, respectively). Solid contours limit the areas with completeness $2$, $10$ and $20$, while dotted lines are reported from Fig. \ref{fig:prob_map} and limit the same detection probability values when no extinction effects are included, thus allowing the direct comparison between the two cases. (*fig:map_extinction*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\Rp}{R_p}$
$\newcommand{\Mp}{M_p}$
$\newcommand{\Rs}{R_{\star}}$
$\newcommand{\aRs}{a/R_{\star}}$
$\newcommand{\JDUTC}{JD_{\textnormal{UTC}}}$
$\newcommand{\RpRs}{\Rp/\Rs}$
$\newcommand{\Rjup}{R_{\textnormal{J}}}$
$\newcommand{\Mjup}{M_{\textnormal{J}}}$
$\newcommand{\Teq}{T_{\textnormal{eq}}}$
$\newcommand{\RE}{R_{\Earth}}$
$\newcommand{\ME}{M_{\Earth}}$
$\newcommand{\RS}{R_{\odot}}$
$\newcommand{\mb}{\mathbf}$
$\newcommand{\gb}{\boldsymbol}$
$\newcommand{\tn}{\textnormal}$
$\newcommand{\um}{\mu\textnormal{m}}$
$\newcommand{\Teff}{T_{\textnormal{eff}}}$
$\newcommand{\FeH}{[{\rm{Fe}/\rm{H}}]}$
$\newcommand{\logg}{\log_{10} g}$
$\newcommand{\XHtwoO}{X_{\rm{H}_2\rm{O}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\Rp$}{R_p}$
$\newcommand{$\Mp$}{M_p}$
$\newcommand{$\Rs$}{R_{\star}}$
$\newcommand{$\aRs$}{a/R_{\star}}$
$\newcommand{$\JDUTC$}{JD_{\textnormal{UTC}}}$
$\newcommand{$\Rp$Rs}{$\Rp$/$\Rs$}$
$\newcommand{$\Rjup$}{R_{\textnormal{J}}}$
$\newcommand{$\Mjup$}{M_{\textnormal{J}}}$
$\newcommand{$\Teq$}{T_{\textnormal{eq}}}$
$\newcommand{$\RE$}{R_{\Earth}}$
$\newcommand{$\ME$}{M_{\Earth}}$
$\newcommand{$\RS$}{R_{\odot}}$
$\newcommand{$\mb$}{\mathbf}$
$\newcommand{$\gb$}{\boldsymbol}$
$\newcommand{$\tn$}{\textnormal}$
$\newcommand{$\um$}{\mu\textnormal{m}}$
$\newcommand{$\Teff$}{T_{\textnormal{eff}}}$
$\newcommand{$\FeH$}{[{\rm{Fe}/\rm{H}}]}$
$\newcommand{$\logg$}{\log_{10} g}$
$\newcommand{$\XHtwoO$}{X_{\rm{H}_2\rm{O}}}$</div>



<div id="title">

# Hubble Space Telescope transmission spectroscopy for the temperate sub-Neptune TOI-270d: a possible hydrogen-rich atmosphere containing water vapour

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.15576-b31b1b.svg)](https://arxiv.org/abs/2211.15576)<mark>Appeared on: 2022-11-29</mark> - _Accepted for publication in AAS journals on November 22, 2022 (received July 5, 2022; revised October 30, 2022)_

</div>
<div id="authors">

<mark>Thomas Mikal-Evans</mark>, et al. -- incl., <mark>Laura Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** TOI-270 d is a temperate sub-Neptune discovered by the Transiting Exoplanet Survey Satellite (TESS) around a bright ($J=9.1$mag) M3V host star. With an approximate radius of$2 $\RE$$and equilibrium temperature of 350 K, TOI-270 d is one of the most promising small exoplanets for atmospheric characterisation using transit spectroscopy. Here we present a primary transit observation of TOI-270 d made with the Hubble Space Telescope (HST) Wide Field Camera 3 (WFC3) spectrograph across the 1.126-1.644$\um$wavelength range, and a 95\%credible upper limit of$8.2 \times 10^{-14}$erg s$^{-1}$cm$^{-2}$\AA$^{-1}$arcsec$^{-2}$for the stellar Ly$\alpha$emission obtained using the Space Telescope Imaging Spectrograph (STIS). The transmission spectrum derived from the TESS and WFC3 data provides evidence for molecular absorption by a hydrogen-rich atmosphere at${4\sigma}$significance relative to a featureless spectrum. The strongest evidence for any individual absorber is obtained for H$_2$O, which is favoured at 3$\sigma$significance. When retrieving on the WFC3 data alone and allowing for the possibility of a heterogeneous stellar brightness profile, the detection significance of H$_2$O is reduced to 2.8$\sigma$. Further observations are therefore required to robustly determine the atmospheric composition of TOI-270 d and assess the impact of stellar heterogeneity. If confirmed, our findings would make TOI-270 d one of the smallest and coolest exoplanets to date with detected atmospheric spectral features.

</div>

<div id="div_fig1">

<img src="tmp_2211.15576/./2d_Image.png" alt="Fig4.1" width="50%"/><img src="tmp_2211.15576/./2d_Image_2.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Calibrated HST STIS Ly$\alpha$ observations of TOI-270. \textit{(Top)} Calibrated spectrum taken on 2020 January 31. Wavelength dispersion direction is along the horizontal axis. The bright stripe down the middle is the Earth's Ly$\alpha$ geocoronal emission. Ly$\alpha$ emission from TOI-270 would be visible alongside this line at the Doppler shift corresponding to the velocity difference between HST and the target. We find no significant evidence of Ly$\alpha$ emission in this exposure. \textit{(Bottom)} Identical as above, but for a second exposure taken on 2020 January 29.  (*fig:STIS_UV*)

</div>
<div id="div_fig2">

<img src="tmp_2211.15576/./limbDarkening_Coefficients.png" alt="Fig6" width="100%"/>

**Figure 6. -** Coefficients ($u_1$, $u_2$) for a quadratic stellar limb darkening law obtained using ExoCTK and PyLDTK. Separate sets of ExoCTK coefficients were computed assuming an ATLAS 9 (blue circles) and Phoenix ACES (red circles) stellar model. The PyLDTK coefficients (black points) assume a Phoenix ACES stellar model. Posterior distributions obtained from the light curve fits are shown as orange diamonds, with $1\sigma$ credible intervals that are smaller than the marker symbols. (*fig:limbDarkeningCoeffs*)

</div>
<div id="div_fig3">

<img src="tmp_2211.15576/./alldata_noSH_spectrum.png" alt="Fig9.1" width="50%"/><img src="tmp_2211.15576/./alldata_SH_spectrum.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Observations and retrieval of the transmission spectrum of TOI-270 d. Top: Retrieval with baseline planetary atmosphere model and no stellar heterogeneity. Bottom: Retrieval including stellar heterogeneity. Both retrievals included the TESS (photometric point at 0.8 $\mu$m) and WFC3 (spectrum between 1.1-1.7 $\mu$m) data, shown in red. The blue curve shows the median-fit model spectrum and the light blue shaded regions show the $1\sigma$ and $2\sigma$ credible ranges. The binned model points for the median-fit spectrum are shown as yellow circles. (*fig:spectrum*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

109  publications in the last 7 days.
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers